In [96]:
#Import dependencies
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd
import time

In [105]:
# Add a start_time variable to measure the execution time
start_time = time.time()
#Create an empty list to store the job postings
id_list = []
start = 0  # Starting point
results_per_page = 10 #How many results will the page show per search
results_desired = 200  #How many results will the script save

# Bucle while to reach the number of jobs desired

while  len(id_list) < results_desired:
    location = "Argentina"  # Job location
   

    # Construct the URL for LinkedIn job search
    list_url = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=&location={location}&geoId=100446943&f_E=2%2C3%2C4&f_PP=103813819%2C117071347%2C102399085&f_TPR=&f_WT=3%2C2&start={start}"


    # Send a GET request to the URL and store the response
    response = requests.get(list_url)

    #Get the HTML, parse the response and find all list items(jobs postings)
    list_data = response.text
    list_soup = BeautifulSoup(list_data, "html.parser")
    page_jobs = list_soup.find_all("li")

 
    #Itetrate through job postings to find job ids
    for job in page_jobs:
        try:
            base_card_div = job.find("div", {"class": "base-card"})
            job_id = base_card_div.get("data-entity-urn").split(":")[3]
        except:
            job_id = 0
        id_list.append(job_id)
        

    start = start + results_per_page
    
end_time = time.time()

time_spent = end_time - start_time

print(time_spent)

31.2921621799469


In [106]:


start_time = time.time()
# Initialize an empty list to store job information
job_list = []

# Loop through the list of job IDs and get each URL
for job_id in id_list:
    # Construct the URL for each job using the job ID
    job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
    
    # Send a GET request to the job URL and parse the reponse
    job_response = requests.get(job_url)
    job_soup = BeautifulSoup(job_response.text, "html.parser")
    
     # Create a dictionary to store job details
    job_post = {}
    
    # Try to extract and store the job title
    try:
        job_post["job_title"] = job_soup.find("h2", {"class":"top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"}).text.strip()
    except:
        job_post["job_title"] = None
        
    # Try to extract and store the company name
    try:
        job_post["company_name"] = job_soup.find("a", {"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
    except:
        job_post["company_name"] = None
        
    # Try to extract and store the time posted
    try:
        job_post["time_posted"] = job_soup.find("span", {"class": "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
    except:
        job_post["time_posted"] = None
        
    # Try to extract and store the number of applicants
#     try:
#         job_post["num_applicants"] = job_soup.find("span", {"class": "num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"}).text.strip()
#     except:
#         job_post["num_applicants"] = None
    
        
    # Append the job details to the job_list
    job_list.append(job_post)

# Create a pandas DataFrame using the list of job dictionaries 'job_list'
jobs_df = pd.DataFrame(job_list)
end_time = time.time()

time_spent = end_time - start_time

print("finished. Time spent:", time_spent)




296.40841126441956
tu lista                                              job_title           company_name  \
0                 Agente de Contact Center Microcentro      BBVA en Argentina   
1                                         ¡¿Te sumás?!        Galicia Seguros   
2    Telefonista Administrativa (Estudiantes de Cs ...            Grupo Abans   
3    ¡Sumate a nuestro equipo de Marketing Perfomance!               Despegar   
4      ¡Sumate a nuestro equipo de Product Management!               Despegar   
..                                                 ...                    ...   
195                         Customer Success Team Lead  Automóviles San Jorge   
196                                               None                   None   
197                       Analista de Recursos Humanos    GLAM Distribuciones   
198             Coordinador de Operaciones Comerciales       Laboratorio Elea   
199                          Desarrollador/a Fullstack      BBVA en Argentina   


In [110]:
start_time = time.time()
jobs_df.to_csv('Linkeding_Scrapping3.csv', index = False)
end_time = time.time()
time_spent = end_time - start_time

print(time_spent)

0.002999544143676758
